In [38]:
!git clone https://github.com/YukinobuYoshihara/yawarakame.git

Cloning into 'yawarakame'...
remote: Enumerating objects: 79, done.
remote: Counting objects: 100% (79/79), done.
remote: Compressing objects: 100% (76/76), done.
remote: Total 79 (delta 39), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (79/79), 405.63 KiB | 2.12 MiB/s, done.
Resolving deltas: 100% (39/39), done.


In [39]:
import os
# 作成されたリポジトリのディレクトリに移動する
os.chdir('yawarakame')

In [40]:
!pip install langchain langchain-openai langchain-community faiss-cpu unstructured

In [41]:
import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")

In [42]:
import os
import random
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.document_loaders import DirectoryLoader, TextLoader, CSVLoader, WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains.history_aware_retriever import create_history_aware_retriever
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage

# (Characterクラスは変更なし)
class Character:
    """キャラクターのRAGチェーンと情報をカプセル化するクラス"""
    def __init__(self, name: str, private_docs_path: str, shared_logs_path: str, system_prompt_template: str, web_urls: list):
        self.name = name
        print(f"キャラクター「{self.name}」を構築中...")
        private_loader = DirectoryLoader(private_docs_path, glob="**/*.txt", loader_cls=TextLoader)
        private_docs = private_loader.load()
        print(f"  > 個別知識を {len(private_docs)} 件読み込みました。")
        shared_docs = []
        character_log_file = os.path.join(shared_logs_path, f"{self.name}.csv")
        if os.path.exists(character_log_file):
            try:
                log_loader = CSVLoader(file_path=character_log_file, encoding='utf-8')
                shared_docs = log_loader.load()
                print(f"  > 共有会話ログ '{os.path.basename(character_log_file)}' を読み込み、{len(shared_docs)} 件のドキュメントを取得しました。")
            except Exception as e:
                print(f"  > 共有会話ログの読み込み中にエラーが発生しました: {e}")
        web_docs = []
        if web_urls:
            print(f"  > Webサイトから最新のファクト情報を読み込みます...")
            try:
                loader = WebBaseLoader(web_urls)
                web_docs = loader.load()
                print(f"  > Webサイトから {len(web_docs)} 件のドキュメントを取得しました。")
            except Exception as e:
                print(f"  > Webサイトの読み込み中にエラーが発生しました: {e}")
        all_docs = private_docs + shared_docs + web_docs
        print(f"  > 合計 {len(all_docs)} 件のドキュメントで知識ベースを構築します。")
        llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.7)
        embeddings = OpenAIEmbeddings()
        text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
        split_docs = text_splitter.split_documents(all_docs)
        vectorstore = FAISS.from_documents(split_docs, embeddings)
        retriever = vectorstore.as_retriever()
        history_aware_prompt = ChatPromptTemplate.from_messages([
            MessagesPlaceholder(variable_name="chat_history"),
            ("user", "{input}"),
            ("user", "上記の発言を踏まえ、関連情報を検索するためのキーワードを生成してください。"),
        ])
        history_aware_retriever = create_history_aware_retriever(llm, retriever, history_aware_prompt)
        answer_prompt = ChatPromptTemplate.from_messages([
            ("system", system_prompt_template),
            MessagesPlaceholder(variable_name="chat_history"),
            ("user", "{input}"),
        ])
        document_chain = create_stuff_documents_chain(llm, answer_prompt)
        self.rag_chain = create_retrieval_chain(history_aware_retriever, document_chain)
        print(f"キャラクター「{self.name}」の構築完了。")
    def speak(self, input_text: str, chat_history: list):
        result = self.rag_chain.invoke({"input": input_text, "chat_history": chat_history})
        return result["answer"]

# ★修正★ DialogueManagerのループ制御を修正
class DialogueManager:
    """対談の進行を管理するクラス"""
    def __init__(self, characters: list, topics: list):
        self.characters = characters
        self.topics = topics
        self.shared_history = []

    def _introduce_topic(self, topic: str):
        print("\n" + "="*50)
        print(f"【新たなテーマ】: {topic}")
        print("="*50 + "\n")
        moderator_message = f"司会者: それでは、次のテーマ「{topic}」について議論を始めましょう。"
        self.shared_history.append(HumanMessage(content=moderator_message))
        return f"最初の議題として、「{topic}」について、皆さんのご意見をお聞かせください。"

    def run_discussion(self, turns_per_character: int = 5):
        for topic in self.topics:
            current_input = self._introduce_topic(topic)
            last_speaker = None
            total_turns = len(self.characters) * turns_per_character
            print(f"今回のテーマでは、合計 {total_turns} 回の発言が予定されています。")
            print("-" * 50 + "\n")
            for turn in range(total_turns):
                if last_speaker:
                    possible_speakers = [c for c in self.characters if c.name != last_speaker.name]
                else:
                    possible_speakers = self.characters

                speaker = random.choice(possible_speakers)
                print(f"--- (ターン{turn + 1}/{total_turns}) {speaker.name}の発言 ---")
                response = speaker.speak(current_input, self.shared_history)

                # ★修正★ AIが議論の終了を提案したかチェック
                if "[NEXT_TOPIC]" in response:
                    print(f"\n... {speaker.name}が次の議題への移行を提案しました。このテーマを終了します。 ...\n")
                    break  # 現在のトピックに関するループを抜ける

                # 通常の応答の場合
                print(response)
                print("-" * 20 + "\n")

                self.shared_history.append(AIMessage(content=response, name=speaker.name))
                current_input = response
                last_speaker = speaker

# ★修正★ メイン処理のプロンプトを大幅に修正
if __name__ == "__main__":
    os.environ["USER_AGENT"] = "MyCustomLangChainBot/1.0"
    script_dir = os.getcwd()

    fact_urls = [
        "https://www.football-lab.jp/nago/",
        "https://www.jleague.jp/standings/j1/"
    ]

    # ★修正★ 指示をより具体的にし、不要な発言を抑制する
    character_definitions = [
        {
            "name": "忍者",
            "private_path": os.path.join(script_dir, "忍者/"),
            "shared_path": os.path.join(script_dir, "conversation_logs/"),
            "prompt": """あなたはJリーグのサッカーに詳しい名古屋グランパスが好きな「忍者」です。
# 指示
- 一人称は「拙者」、語尾は「～でござる」「ﾆﾝﾆﾝ」を基本とします。
- 発言はユーモアにあふれ、あたたかでファン目線の応援基調での視点で行われます。
- **最重要：** 発言の冒頭で名乗ったり、「意見を述べる」といった前置きは絶対にしないでください。
- **最重要：** 他のキャラクターの発言を評価するだけの導入（「〇〇の意見は的確だ」など）は避け、すぐにあなたの意見や分析を述べてください。
- 他のキャラクターの口調に影響されず、あなた自身の役割（忍者、ござる口調）を厳密に守ってください。
- 提供されたWebサイトの情報（最新の順位表やチームのスタッツ）を積極的に利用し、具体的でデータに基づいた発言をしてください。
- あなたの発言は常に200文字程度で、要点をまとめて簡潔に話してください。
- ★追加ルール★ このテーマの議論が尽きたと感じ、次のテーマに進むべきだと判断した場合は、他の言葉は一切含めず、必ず `[NEXT_TOPIC]` とだけ応答してください。
{context}"""
        },
        {
            "name": "侍",
            "private_path": os.path.join(script_dir, "侍/"),
            "shared_path": os.path.join(script_dir, "conversation_logs/"),
            "prompt": """あなたはJリーグの名古屋グランパスが好きな「侍」です。
# 指示
- 一人称は「侍」、「～であろう」「～なかろう」といった武士を思わせる言葉遣いで話します。
- 発言は冷静で、かなり厳しめで批判的なトーンでの分析的な視点で行われます。
- **最重要：** 発言の冒頭で名乗ったり、「意見を述べる」といった前置きは絶対にしないでください。
- **最重要：** 他のキャラクターの発言を評価するだけの導入（「〇〇の意見は的確だ」など）は避け、すぐにあなたの意見や分析を述べてください。
- 他のキャラクターの口調に影響されず、あなた自身の役割（侍、武士口調）を厳密に守ってください。
- 提供されたWebサイトの情報（最新の順位表やチームのスタッツ）を積極的に利用し、具体的でデータに基づいた発言をしてください。
- あなたの発言は常に200文字程度で、要点をまとめて簡潔に話してください。
- ★追加ルール★ このテーマの議論が尽きたと感じ、次のテーマに進むべきだと判断した場合は、他の言葉は一切含めず、必ず `[NEXT_TOPIC]` とだけ応答してください。
{context}"""
        },
        {
            "name": "記者",
            "private_path": os.path.join(script_dir, "記者/"),
            "shared_path": os.path.join(script_dir, "conversation_logs/"),
            "prompt": """あなたはJリーグの名古屋グランパスが好きな記者です。
# 指示
- 口調は「ですます」体を基本とします。
- 役割は対談のファシリテーターです。
- テーマでの最後の発言は、そのテーマでの他のメンバーの発言をまとめるものにしてください。
- **最重要：** 発言の冒頭で名乗ったり、「質問します」といった前置きは絶対にしないでください。
- **最重要：** 他のキャラクターの発言を評価するだけの導入（「〇〇の意見は的確だ」など）は避け、すぐにあなたの意見や分析、または具体的な質問を述べてください。
- 他のキャラクターの口調に影響されず、あなた自身の役割（記者、ですます調）を厳密に守ってください。
- テーマについて、Webサイトから得られる客観的なデータ（順位、スタッツ等）を基に、侍や忍者に対して鋭い質問を投げかけて議論を深めてください。
- あなたの発言は常に200文字程度で、要点をまとめて簡潔に話してください。
- ★追加ルール★ このテーマの議論が尽きたと感じ、次のテーマに進むべきだと判断した場合は、他の言葉は一切含めず、必ず `[NEXT_TOPIC]` とだけ応答してください。
{context}"""
        }
    ]

    today_str = "2025年7月31日"
    discussion_topics = [
        f"{today_str}時点での名古屋グランパスの成績（順位、スタッツ）をどう評価するか？",
        "現在のチームの強みと弱点をデータからどう分析するか？",
        "シーズン後半戦、順位を上げるために何が必要か？"
    ]

    print("対談シミュレーションを開始します...")
    characters = [
        Character(
            name=c["name"],
            private_docs_path=c["private_path"],
            shared_logs_path=c["shared_path"],
            system_prompt_template=c["prompt"],
            web_urls=fact_urls
        )
        for c in character_definitions
    ]

    manager = DialogueManager(characters, discussion_topics)
    manager.run_discussion(turns_per_character=5)
    print("\n対談シミュレーションを終了します。")

対談シミュレーションを開始します...
キャラクター「忍者」を構築中...
  > 個別知識を 1 件読み込みました。
  > 共有会話ログ '忍者.csv' を読み込み、174 件のドキュメントを取得しました。
  > Webサイトから最新のファクト情報を読み込みます...
  > Webサイトから 2 件のドキュメントを取得しました。
  > 合計 177 件のドキュメントで知識ベースを構築します。
キャラクター「忍者」の構築完了。
キャラクター「侍」を構築中...
  > 個別知識を 1 件読み込みました。
  > 共有会話ログ '侍.csv' を読み込み、62 件のドキュメントを取得しました。
  > Webサイトから最新のファクト情報を読み込みます...
  > Webサイトから 2 件のドキュメントを取得しました。
  > 合計 65 件のドキュメントで知識ベースを構築します。
キャラクター「侍」の構築完了。
キャラクター「記者」を構築中...
  > 個別知識を 1 件読み込みました。
  > 共有会話ログ '記者.csv' を読み込み、228 件のドキュメントを取得しました。
  > Webサイトから最新のファクト情報を読み込みます...
  > Webサイトから 2 件のドキュメントを取得しました。
  > 合計 231 件のドキュメントで知識ベースを構築します。
キャラクター「記者」の構築完了。

【新たなテーマ】: 2025年7月31日時点での名古屋グランパスの成績（順位、スタッツ）をどう評価するか？

今回のテーマでは、合計 15 回の発言が予定されています。
--------------------------------------------------

--- (ターン1/15) 記者の発言 ---
2025年7月31日時点で名古屋グランパスはJ1リーグで15位に位置しています。過去数年の成績を振り返ると、2023年は6位、2024年は11位と、明らかに成績が下降しています。特に攻撃面では、ゴール期待値が1.153に対し、実際のゴール数は1.17とほぼ同じですが、被ゴール数は1.33と期待値を上回っています。この点について、どのように改善策を考えますか？また、選手のパフォーマンスや戦術についてもお聞きしたいです。
---------

KeyboardInterrupt: 